# Preprocessing and Feature Creation

In this notebook we import the data, preprocess the data and create features for supervised and unsupervised cross-lingual-information retrieval models.

## I. Import Data

In this section we import the English and German europarl datasets and combine them into a parallel sentence translation dataframe.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data import create_data_subset

In [3]:
create_data_subset(sentence_data_source_path='../data/external/europarl-v7.it-en.en',
                   sentence_data_target_path='../data/external/europarl-v7.it-en.it',
                   sample_size=500,
                   sentence_data_sampled_path="../data/interim/europarl_en_it.pkl",)

Finished function: 'load_doc' in 1.34 seconds.
Finished function: 'to_sentences' in 0.71 seconds.
Finished function: 'load_doc' in 1.89 seconds.
Finished function: 'to_sentences' in 0.97 seconds.
Sampled dataframe saved in: ../data/interim/europarl_en_it.pkl
Finished function: 'create_data_subset' in 6.26 seconds.


## II. Preprocess data

In this section we preprocess the parallel sentence data for the feature generation.

In [3]:
import spacy
from nltk.corpus import stopwords
from textblob import TextBlob as textblob_source
from textblob_de import TextBlobDE as textblob_target
#import en_core_web_sm
# import de_core_news_sm
#import it_core_news_sm
# import pl_core_news_sm
import time

In [ ]:
from src.data import PreprocessingEuroParl

In [7]:
stopwords_source = stopwords.words('english')
stopwords_target = stopwords.words('german') # German stopwords
#stopwords_target = stopwords.words('italian') # Italian stopwords
# stopwords_target = stopwords.words('polish') # Polish stopwords
#nlp_source = en_core_web_sm.load()
# nlp_target = de_core_news_sm.load() # German pipeline
#nlp_target = it_core_news_sm.load() # Italian pipeline
# nlp_target = pl_core_news_sm.load() # Polish pipeline

In [7]:
parallel_sentences = PreprocessingEuroParl(df_sampled_path="../data/interim/europarl_en_it.pkl")

Finished function: 'import_data' in 0.0 seconds.


In [ ]:
stopw

In [9]:
parallel_sentences.preprocess_sentences(stopwords_source, stopwords_target, nlp_source, nlp_target)

  0%|          | 0/500 [00:00<?, ?it/s]

Finished function: 'lemmatize' in 3.12 seconds.
Finished function: 'tokenize_sentence' in 0.07 seconds.
Finished function: 'strip_whitespace' in 0.0 seconds.
Finished function: 'lowercase' in 0.0 seconds.
Finished function: 'remove_punctuation' in 0.0 seconds.
Finished function: 'remove_stopwords' in 0.02 seconds.
Finished function: 'remove_numbers' in 0.0 seconds.
Finished function: 'create_cleaned_token_embedding' in 3.22 seconds.


  0%|          | 0/500 [00:00<?, ?it/s]

Finished function: 'lemmatize' in 2.63 seconds.
Finished function: 'tokenize_sentence' in 0.07 seconds.
Finished function: 'strip_whitespace' in 0.0 seconds.
Finished function: 'lowercase' in 0.0 seconds.
Finished function: 'remove_punctuation' in 0.0 seconds.
Finished function: 'remove_stopwords' in 0.03 seconds.
Finished function: 'remove_numbers' in 0.01 seconds.
Finished function: 'create_cleaned_token_embedding' in 2.74 seconds.
Finished function: 'tokenize_sentence' in 0.07 seconds.
Finished function: 'strip_whitespace' in 0.0 seconds.
Finished function: 'lowercase' in 0.0 seconds.
Finished function: 'create_cleaned_text' in 0.07 seconds.


100%|██████████| 500/500 [00:00<00:00, 273672.45it/s]

Finished function: 'tokenize_sentence' in 0.07 seconds.
Finished function: 'strip_whitespace' in 0.0 seconds.
Finished function: 'lowercase' in 0.0 seconds.
Finished function: 'create_cleaned_text' in 0.08 seconds.


In [ ]:
parallel_sentences.extract_sentence_information(nlp_source, nlp_target, textblob_source, textblob_target)

100%|██████████| 500/500 [00:00<00:00, 90141.93it/s]
/Users/jakob/PycharmProjects/crosslingual-information-retrieval/src/data/preprocess_data.py:243: RuntimeWarning: divide by zero encountered in log
  return (character_vector / word_vector).replace(np.nan, 0).replace(np.inf, 0).replace(np.log(0), 0)
100%|██████████| 500/500 [00:00<00:00, 370456.10it/s]

Finished function: 'number_punctuations_total' in 0.01 seconds.
Finished function: 'number_punctuations_total' in 0.01 seconds.
Finished function: 'number_words' in 0.0 seconds.
Finished function: 'number_words' in 0.0 seconds.
Finished function: 'number_unique_words' in 0.01 seconds.
Finished function: 'number_unique_words' in 0.01 seconds.
Finished function: 'number_characters' in 0.01 seconds.
Finished function: 'number_characters' in 0.01 seconds.
Finished function: 'average_characters' in 0.0 seconds.
Finished function: 'average_characters' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds


  0%|          | 0/500 [00:00<?, ?it/s]

Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished function: 'number_punctuation_marks' in 0.0 seconds.
Finished

  3%|▎         | 16/500 [00:00<00:03, 159.99it/s]

...

  5%|▍         | 23/500 [00:00<00:02, 217.16it/s]

Finished function: 'number_times' in 2.47 seconds.


  4%|▍         | 22/500 [00:00<00:02, 212.63it/s]

Finished function: 'number_times' in 2.87 seconds.


 89%|████████▉ | 446/500 [00:02<00:00, 178.81it/s]

In [ ]:
parallel_sentences.create_embedding_information("proc_5k")

In [ ]:
parallel_sentences.create_embedding_information("proc_b_1k")

In [ ]:
parallel_sentences.create_embedding_information("vecmap")

In [ ]:
parallel_sentences.preprocessed.to_json("../data/interim/preprocessed_data_en_it.json")

In [ ]:
# import pandas as pd
# preprocessed_data = pd.read_json("../data/interim/preprocessed_data.json")

## III. Create data set

In this section we create the datasets for the training of the supervised model and the data for the supervised and unsupervised retrieval.

In [ ]:
%autoreload 2
from src.data import DataSet

In [ ]:
n_model = 0
n_queries = 100
n_retrieval = 500
k = 0
sample_size_k = 100

In [ ]:
dataset = DataSet(parallel_sentences.preprocessed)

In [ ]:
dataset.split_model_retrieval(n_model, n_retrieval)

In [ ]:
# dataset.create_model_index(n_model, k, sample_size_k,
#     "sentence_embedding_tf_idf_proc_5k_source", "sentence_embedding_tf_idf_proc_5k_target")

In [ ]:
# dataset.model_dataset_index.reset_index(drop=True).to_feather("../data/processed/dataset_model_index_en_it.feather")

In [ ]:
# import pandas as pd
# pd.read_feather("../data/processed/dataset_model_index.feather")

In [ ]:
dataset.create_retrieval_index(n_queries)

In [ ]:
dataset.retrieval_dataset_index.reset_index(drop=True).to_feather("../data/processed/dataset_retrieval_index_en_it.feather")

In [ ]:
# import pandas as pd
# pd.read_feather("../data/processed/dataset_retrieval_index.feather")

## IV. Create features

In this section we create features for our model, that are sentence based and should be created before the text is preprocessed.

In [ ]:
%autoreload 2
from src.features import feature_generation_class

In [ ]:
# import pickle
# with open(r"../data/processed/correlated_features.pkl", "rb") as file:
#    chosen_features = pickle.load(file)

Generation of the training data for the supervised classifciation model.

In [ ]:
# features_model = feature_generation_class.FeatureGeneration(dataset.model_dataset_index, 
#                                                             parallel_sentences.preprocessed)

In [ ]:
# features_model.create_feature_dataframe()

In [ ]:
# features_model.create_sentence_features()

In [ ]:
# features_model.create_embedding_features("proc_5k")

In [ ]:
features_model.create_embedding_features("proc_b_1k")

In [ ]:
# features_model.create_embedding_features("vecmap")

In [ ]:
# features_model.feature_dataframe.reset_index(drop=True).to_feather("../data/processed/feature_model_en_it.feather")

In [ ]:
# import pandas as pd
# pd.read_feather("../data/processed/feature_model.feather")

Generation of the data for the crosslingual information retrieval task.

In [ ]:
features_retrieval = feature_generation_class.FeatureGeneration(dataset.retrieval_dataset_index, 
                                                            parallel_sentences.preprocessed)

In [ ]:
features_retrieval.create_feature_dataframe()

In [ ]:
features_retrieval.create_sentence_features()

In [ ]:
features_retrieval.create_embedding_features("proc_5k")

In [ ]:
features_retrieval.create_embedding_features("proc_b_1k")

In [ ]:
features_retrieval.create_embedding_features("vecmap")

In [ ]:
features_retrieval.feature_dataframe.reset_index(drop=True).to_feather("../data/processed/feature_retrieval_en_it.feather")

In [ ]:
# import pandas as pd
# pd.read_feather("../data/processed/feature_retrieval.feather")